In [0]:
from textblob import TextBlob
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
analysis = TextBlob("This table is black")
print(analysis.sentiment)

print(analysis.tags)

print(analysis.translate(to='es'))

print(dir(analysis))

Sentiment(polarity=-0.16666666666666666, subjectivity=0.43333333333333335)
[('This', 'DT'), ('table', 'NN'), ('is', 'VBZ'), ('black', 'JJ')]
Esta mesa es negra
['__add__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cmpkey', '_compare', '_create_sentence_objects', '_strkey', 'analyzer', 'classifier', 'classify', 'correct', 'detect_language', 'ends_with', 'endswith', 'find', 'format', 'index', 'join', 'json', 'lower', 'ngrams', 'noun_phrases', 'np_counts', 'np_extractor', 'parse', 'parser', 'polarity', 'pos_tagger', 'pos_tags', 'raw', 'raw_sentences', 'replace', 'rfind', 'rindex', 'sentences', 'sentiment', 'sentiment_assessments', 'seria

In [0]:
from google.colab import files
import io

positivas = files.upload()
positivas
pos_count = 0
pos_correct = 0
with open("positivas.txt","r") as f:
  for line in f.read().split('\n'):
    analysis = TextBlob(line)
    #print(line)
    try:
      eng=analysis.translate(to='en')
      if eng.sentiment.polarity > 0:
        pos_correct += 1
      pos_count +=1
    except:
      print ("El elemento no está presente")



print("Precisión positiva = {}% via {} ejemplos".format(pos_correct/pos_count*100.0, pos_count))



MessageError: ignored

In [0]:
from google.colab import files
import io

negativas = files.upload()
negativas
neg_count = 0
neg_correct = 0

with open("negativas.txt","r") as f:
  for line in f.read().split('\n'):
    analysis = TextBlob(line)
    #print(line)
    try:
      eng=analysis.translate(to='en')
      if eng.sentiment.polarity <= 0:
        neg_correct += 1
      neg_count +=1
    except:
      print('el elemento no esta presente')

print("Precisión negativa = {}% via {} ejemplos".format(neg_correct/neg_count*100.0, neg_count))



Saving negativas.txt to negativas (1).txt
Precisión negativa = 90.0% via 20 ejemplos


In [0]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener


ckey="F2pIrutjymGr9vZuqTeViAymw"
csecret="QFLTXwFJZNuR6i00IswAZIgaKsKl5AtmPufoSaRnR57ER2yVxS"
atoken="879408865997201408-QJDeVNKYBsTdp97caK0qFV454YYwRLp"
asecret="gLNfeVRjT7LrDaWWSgCX0ZRu6TeTPBquTxlYKQ4hUzAka"

class listener(StreamListener):

  def on_data(self, data):
    print(data)
    return(True)

  def on_error(self, status):
    print (status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
#Busca la palabra python

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["python"])


In [11]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import sqlite3
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from unidecode import unidecode
import time

analyzer = SentimentIntensityAnalyzer()

ckey="F2pIrutjymGr9vZuqTeViAymw"
csecret="QFLTXwFJZNuR6i00IswAZIgaKsKl5AtmPufoSaRnR57ER2yVxS"
atoken="879408865997201408-QJDeVNKYBsTdp97caK0qFV454YYwRLp"
asecret="gLNfeVRjT7LrDaWWSgCX0ZRu6TeTPBquTxlYKQ4hUzAka"

conn = sqlite3.connect('twitterF1.db')
c = conn.cursor()

def create_table():
  try:
    c.execute("CREATE TABLE IF NOT EXISTS sentimiento(unix REAL, tweet TEXT, sentimiento REAL)")
    c.execute("CREATE INDEX fast_unix ON sentimiento(unix)")
    c.execute("CREATE INDEX fast_tweet ON sentimiento(tweet)")
    c.execute("CREATE INDEX fast_sentimiento ON sentimiento(sentimiento)")
    conn.commit()
  except Exception as e:
    print(str(e))
create_table()

class listener(StreamListener):

  def on_data(self, data):
    try:
      data = json.loads(data)
      tweet = unidecode(data['text'])
      time_ms = data['timestamp_ms']
      vs = analyzer.polarity_scores(tweet)
      sentimiento = vs['compound']
      print(time_ms, tweet, sentimiento)
      c.execute("INSERT INTO sentimiento (unix, tweet, sentimiento) VALUES (?, ?, ?)",
            (time_ms, tweet, sentimiento))
      conn.commit()

    except KeyError as e:
      print(str(e))
    return(True)

  def on_error(self, status):
    print(status)

while True:

  try:
    auth = OAuthHandler(ckey, csecret)
    auth.set_access_token(atoken, asecret)
    twitterStream = Stream(auth, listener())
    twitterStream.filter(track=["a","e","i","o","u"])
  except Exception as e:
    print(str(e))
    time.sleep(5)

ModuleNotFoundError: ignored

In [0]:
import pan

In [10]:

import sqlite3
import pandas as pd

conn = sqlite3.connect('twitterF1.db')
c = conn.cursor()

df = pd.read_sql("SELECT * FROM sentimiento WHERE tweet LIKE '%py%' ORDER BY unix DESC LIMIT 1000", conn)
df.sort_values('unix', inplace=True)

df['sentimiento_smoothed'] = df['sentimiento'].rolling(int(len(df)/5)).mean()
df.dropna(inplace=True)
print(df.tail())


DatabaseError: ignored